# Users and Variance

This notebook will analyse the arrivals of users (whether it is linear or not) and how it affects the variance and the mean of the metrics

In [2]:
from __future__ import annotations
from IPython import get_ipython
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

import scipy.stats as stats

from sklearn.metrics import auc

In [29]:
base_path = os.path.abspath('')

events_df = pd.read_csv(f"{base_path}/Data/2019-Oct.csv")

events_df['time'] = pd.to_datetime(events_df['event_time'])
events_df['date'] = events_df['time'].dt.date
events_df["sales"] = np.where(events_df["event_type"]=="purchase", events_df["price"], 0)
events_df = pd.get_dummies(data=events_df, columns=["event_type"], dtype=float)
events_df.head()

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session,time,date,sales,event_type_cart,event_type_purchase,event_type_remove_from_cart,event_type_view
0,2019-10-01 00:00:00 UTC,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,2019-10-01 00:00:00+00:00,2019-10-01,0.0,1.0,0.0,0.0,0.0
1,2019-10-01 00:00:03 UTC,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,2019-10-01 00:00:03+00:00,2019-10-01,0.0,1.0,0.0,0.0,0.0
2,2019-10-01 00:00:07 UTC,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,2019-10-01 00:00:07+00:00,2019-10-01,0.0,1.0,0.0,0.0,0.0
3,2019-10-01 00:00:07 UTC,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,2019-10-01 00:00:07+00:00,2019-10-01,0.0,1.0,0.0,0.0,0.0
4,2019-10-01 00:00:15 UTC,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,2019-10-01 00:00:15+00:00,2019-10-01,0.0,1.0,0.0,0.0,0.0


In [4]:
# # Example DataFrame
# df = pd.DataFrame({'date': ["2020-01-01", "2020-01-01", "2020-01-01", "2020-01-01", 
#                             "2020-01-02", "2020-01-02", 
#                             "2020-01-03", "2020-01-03", "2020-01-03", 
#                             "2020-01-04", "2020-01-04", "2020-01-04"],
#                    'user_id': [1, 2, 3, 4, 1, 2, 3, 4, 5, 6, 5, 6],
#                    'metric1': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
#                    'metric2': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]})
# #df = df.set_index('date')
# data = pd.DataFrame()

# for d in df["date"].unique():
#     this_df = df[df["date"] <= d]
#     aggreg = (this_df
#            .groupby("user_id")
#            .agg({
#                 "metric1":"sum",
#                 "metric2":"sum",
#            })
#            ).reset_index()
#     aggreg["metric1_sq"] = aggreg["metric1"]**2
#     aggreg["metric2_sq"] = aggreg["metric2"]**2
#     #print(aggreg)
#     aggreg = (
#         aggreg
#         .agg({
#             "user_id":"count",
#             "metric1":"sum",
#             "metric2":"sum",
#             "metric1_sq":"sum",
#             "metric2_sq":"sum",
#         })
#         )
#     this_line=pd.DataFrame({"date":[d], "cum_users": [aggreg[0]], 
#                             "m1_mean":[aggreg[1]/aggreg[0]],
#                             "m1_var":[(aggreg[3]/aggreg[0])-(aggreg[1]/aggreg[0])**2],
#                             "m2_mean":[aggreg[2]/aggreg[0]],
#                             "m2_var":[(aggreg[4]/aggreg[0])-(aggreg[2]/aggreg[0])**2],
#                             "m1_sum":[aggreg[1]], "m1_sum_sq":[aggreg[3]],
#                             "m2_sum":[aggreg[2]], "m2_sum_sq":[aggreg[4]]})
#     data = pd.concat([data, this_line])
# data.reset_index(drop=True)

,date,cum_users,m1_mean,m1_var,m2_mean,m2_var,m1_sum,m1_sum_sq,m2_sum,m2_sum_sq
0,2020-01-01,4,6.50,1.250000,16.50,1.250000,26,174,66,1094
1,2020-01-02,4,11.25,14.687500,26.25,77.187500,45,565,105,3065
2,2020-01-03,5,16.20,6.560000,34.20,35.360000,81,1345,171,6025
3,2020-01-04,6,21.00,35.666667,41.00,35.666667,126,2860,246,10300


In [49]:
cum_users = pd.DataFrame()

for d in events_df["date"].unique()[:40]:
    this_df = events_df[events_df["date"] <= d]
    users_df = (this_df
           .groupby("user_id")
           .agg({
                "event_type_purchase":          ["sum", "max"],
                "sales":                        ["sum"],
           })
           ).reset_index()
    users_df.columns = ["user_id", "n_purchases", "has_purchase", "sales"]
    for c in users_df.columns[1:]:
        users_df[f"{c}_sq"] = users_df[c]**2
    aggregated = (
        users_df
        .agg({
            "user_id":"count",
            "n_purchases":"sum",
            "n_purchases_sq":"sum",
            "has_purchase":"sum",
            "sales":"sum",
            "sales_sq":"sum",
        })
        )
    this_line=pd.DataFrame({"date":[d], "cum_users": [aggregated["user_id"]], 
                            "n_purchases_mean":[aggregated["n_purchases"]/aggregated["user_id"]],
                            "n_purchases_sd":[((aggregated["n_purchases_sq"]/aggregated["user_id"])-(aggregated["n_purchases"]/aggregated["user_id"])**2)**(1/2)],
                            "has_purchase_mean":[aggregated["has_purchase"]/aggregated["user_id"]],
                            "has_purchase_sd":[((aggregated["has_purchase"]/aggregated["user_id"])-(aggregated["has_purchase"]/aggregated["user_id"])**2)**(1/2)],
                            "sales_mean":[aggregated["sales"]/aggregated["user_id"]],
                            "sales_sd":[((aggregated["sales_sq"]/aggregated["user_id"])-(aggregated["sales"]/aggregated["user_id"])**2)**(1/2)],
                            })
    cum_users = pd.concat([cum_users, this_line])
cum_users.reset_index(drop=True)

,date,cum_users,n_purchases_mean,n_purchases_sd,has_purchase_mean,has_purchase_sd,sales_mean,sales_sd
0,2019-10-01,19230.0,0.440770,2.742626,0.052054,0.222136,2.261943,14.023599
1,2019-10-02,50969.0,0.344837,2.525474,0.039946,0.195832,1.750934,12.734046
2,2019-10-03,64325.0,0.411053,2.742456,0.047695,0.213121,2.061785,13.650312
3,2019-10-04,75763.0,0.448807,2.864010,0.051885,0.221796,2.224193,14.061465
4,2019-10-05,87760.0,0.455139,2.881026,0.052530,0.223093,2.253191,14.239498
5,2019-10-06,115609.0,0.408342,2.740920,0.046813,0.211238,1.997317,13.338409
6,2019-10-07,137427.0,0.411739,2.795368,0.047072,0.211793,2.021040,13.477598
7,2019-10-08,152695.0,0.426916,2.858115,0.048770,0.215388,2.100665,13.783683
8,2019-10-09,165609.0,0.444734,2.903871,0.050861,0.219713,2.202852,14.090133
9,2019-10-10,177414.0,0.460894,2.978186,0.052572,0.223177,2.293110,14.582364


In [6]:
cum_users

Index(['event_time', 'product_id', 'category_id', 'category_code', 'brand',
       'price', 'user_id', 'user_session', 'time', 'date', 'sales',
       'event_type_cart', 'event_type_purchase', 'event_type_remove_from_cart',
       'event_type_view'],
      dtype='object')

In [18]:
# Transform to user-based data
users_df = events_df.groupby("date").agg(
    {
        "user_id":                      ['size'],
        "sales":                        ['sum', "nunique"],
        # "event_type_view":              ["sum", "max"],
        # "event_type_cart":              ["sum", "max"],
        # "event_type_remove_from_cart":  ["sum", "max"],
        # "event_type_purchase":          ["sum", "max"],
        # "date":                         ["min", "max"]
    }
)
# users_df.columns = ['_'.join(col).strip() for col in users_df.columns.values]
# users_df = users_df.rename(columns={
#     'user_id_size': 'num_events', 'sales_sum': 'total_sales', 
#     'event_type_view_sum': 'total_views', 'event_type_view_max': 'has_view',
#     'event_type_cart_sum': 'total_cart', 'event_type_cart_max': 'has_cart',
#     'event_type_remove_from_cart_sum': 'total_remove_from_cart', 'event_type_remove_from_cart_max': 'has_remove_from_cart',
#     'event_type_purchase_sum': 'total_purchases', 'event_type_purchase_max': 'has_purchases',
# }).reset_index()
users_df


user_id     sales        
              size       sum nunique
date                                
2019-10-01  142414  43497.17     573
2019-10-02  201068  45746.20     592
2019-10-03  124847  43380.98     606
2019-10-04  115612  35887.15     527
2019-10-05  106343  29228.55     503
2019-10-06  187383  33167.80     523
2019-10-07  181451  46837.58     601
2019-10-08  148944  43015.61     592
2019-10-09  139901  44051.02     581
2019-10-10  132634  42017.69     593
2019-10-11  119195  36024.56     548
2019-10-12  111166  32269.39     512
2019-10-13  116755  35686.30     546
2019-10-14  133258  40785.22     564
2019-10-15  130030  40471.08     564
2019-10-16  136043  42755.28     589
2019-10-17  133870  45325.16     573
2019-10-18  119073  39829.67     591
2019-10-19  104300  29598.43     506
2019-10-20  114682  34260.94     557
2019-10-21  139662  45934.86     592
2019-10-22  136370  42496.42     577
2019-10-23  133764  42503.91     599
2019-10-24  129841  43363.30     555
2019-10-25  111774  32350.47     528
2019-10-26  109133  25359.18     463
2019-10-27  127318  37083.45     544
2019-10-28  132132  40842.98     579
2019-10-29  134769  39202.60     553
2019-10-30  125597  38832.06     566
2019-10-31  122954  39733.42     602